<a href="https://colab.research.google.com/github/JuniorHZ19/ML/blob/main/perceptronmulticapa3.5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision

In [2]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler



class MiDataSet(Dataset):

  def __init__(self,csv_file):

     data=pd.read_csv(csv_file)

     self.x,self.y= self.limpiezaDatos(data)

     self.samples=self.y.shape[0]


  def __getitem__(self,id):

   return self.x[id],self.y[id]


  def __len__(self):
   return self.samples


  def limpiezaDatos(self,data):

    data=data.dropna(how="all")

    data=data.drop_duplicates()

    data=data.drop(columns=["name","ticket","cabin","boat","body","home.dest"])

    data[["age","fare"]]=data[["age","fare"]].fillna(data[["age","fare"]].mean())

    data["embarked"]=data["embarked"].fillna("S")

    data["age"]=data["age"].astype(int)

    data=pd.get_dummies (data,columns=["sex","embarked"],prefix=["sex","embarked"])

    variables_x=data.drop(columns=["survived"])

    print(variables_x.head())

    print(data["survived"].head())
    self.scaler=StandardScaler()

    variables_x_esclados=self.scaler.fit_transform(variables_x)

    variable_y=data["survived"].values


    datos_x=torch.from_numpy(variables_x_esclados).float()

    datos_y=torch.from_numpy(variable_y).float()



    return datos_x,  datos_y







In [11]:
import sklearn

from sklearn.model_selection import train_test_split

mis_datos=MiDataSet("/content/titanic.csv")



X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(mis_datos.x, mis_datos.y, test_size=0.2, random_state=0)

print(X_entrenamiento.shape)

print(X_prueba.shape)

print(y_entrenamiento.shape)

print(y_prueba.shape)




   pclass  age  sibsp  parch      fare  sex_female  sex_male  embarked_C  \
0       1   29      0      0  211.3375           1         0           0   
1       1    0      1      2  151.5500           0         1           0   
2       1    2      1      2  151.5500           1         0           0   
3       1   30      1      2  151.5500           0         1           0   
4       1   25      1      2  151.5500           1         0           0   

   embarked_Q  embarked_S  
0           0           1  
1           0           1  
2           0           1  
3           0           1  
4           0           1  
0    1
1    1
2    0
3    0
4    0
Name: survived, dtype: int64
torch.Size([1047, 10])
torch.Size([262, 10])
torch.Size([1047])
torch.Size([262])


In [12]:



from torch.utils.data import DataLoader, TensorDataset


conjunto_entrenamiento = TensorDataset(X_entrenamiento, y_entrenamiento)

conjunto_prueba = TensorDataset(X_prueba, y_prueba)



data_por_lote_entrenamiento=DataLoader(conjunto_entrenamiento,batch_size=100,shuffle=True)

data_por_por_lote_test=DataLoader(conjunto_prueba,batch_size=100,shuffle=True)



datos_features_train,datos_label_train=next(iter(data_por_lote_entrenamiento))

datos_features_test,datos_label_test=next(iter(data_por_por_lote_test))


print(datos_features_test.shape)




torch.Size([100, 10])


In [6]:
import torch.nn as nn
import torch.nn.functional as F

modelo=torch.Sequential(

                        torch.nn.Linear(input_size,3),
                        torch.nn.ReLU(),

                        torch.nn.Linear(3,4),
                        torch.nn.ReLU(),

                        torch.nn.Linear(4,1),
                        torch.nn.Sigmoid()
)

AttributeError: ignored

In [5]:
import torch.nn as nn
import torch.nn.functional as F



class PerceptronMulticapa(nn.Module):

   def __init__(self,input_size):
    super(PerceptronMulticapa,self).__init__()

    self.capa_oculta1=nn.Linear(input_size,1000)

    self.capa_oculta2=nn.Linear(1000,500)

    self.capa_oculta3=nn.Linear(500,200)

    self.capa_salida =nn.Linear(200,1)

   def forward(self,x):

     x=self.capa_oculta1(x)
     x=F.relu(x)

     x=self.capa_oculta2(x)
     x=F.relu(x)

     x=self.capa_oculta3(x)
     x=F.relu(x)

     x=self.capa_salida(x)
     output=torch.sigmoid(x)


     return output


In [20]:

if torch.cuda.is_available():
    print("CUDA está disponible.")
    device = torch.device("cuda")
else:
    print("CUDA no está disponible. Se utilizará la CPU.")
    device = torch.device("cpu")

CUDA está disponible.


In [33]:



#funcion clasificador:
def clasificador(y_pred_test):

  umbral = 0.6
  y_pred_test_binario = (y_pred_test >= umbral).to(torch.float32)

  return y_pred_test_binario

#funcion acutity:
def accurity(y_test,y_real):
   correctos=0
   for predicho, real in zip(y_test, y_real):
    if predicho==real:
     correctos+=1
   return correctos/len(y_test)


import torch.optim as optim



def TrainMLP(modelo,epochs,optimisador,criterio,input_size,data_por_lote_entrenamiento,data_por_por_lote_test):

 for epoch in range(epochs):
  for x_entrenamiento,y_entrenamiento in (data_por_lote_entrenamiento):
   total_train_loss = 0.0
   num_train_batches = len(data_por_lote_entrenamiento)
   modelo.train()
   x_entrenamiento,y_entrenamiento=x_entrenamiento.to(device),y_entrenamiento.to(device)

   y_predict=PerceptronMulti(x_entrenamiento)
   loss=criterio(y_predict,y_entrenamiento.view(-1,1))
   loss.backward()
   optimizer.step()
   optimizer.zero_grad()
   total_train_loss+=loss.item()

   modelo.eval()
   with torch.no_grad():
    for X_prueba,y_prueba in (data_por_por_lote_test):
     total_test_accuracy = 0.0
     num_test_batches = len(data_por_por_lote_test)
     X_prueba,y_prueba=X_prueba.to(device),y_prueba.to(device)
     y_predict_test=PerceptronMulti(X_prueba)
     y_predic_test_clasificado=clasificador(y_predict_test)
     Precision=accurity(y_prueba,y_predic_test_clasificado)
     total_test_accuracy += Precision
  avg_test_accuracy = total_test_accuracy / num_test_batches
  avg_train_loss=total_train_loss/num_train_batches
  print(f'Epoca[{epoch+1}], Perdida: {avg_train_loss:.4f},Accuryty:{avg_test_accuracy:.4f}')





#-----------------------------------------------------------------------------------------------

input_size=X_entrenamiento.shape[1]

PerceptronMulti=PerceptronMulticapa(input_size)

criterio=nn.BCELoss()

optimizer=optim.Adam(PerceptronMulti.parameters(),lr=0.0000001)

epochs=500

PerceptronMulti=PerceptronMulti.to(device)

TrainMLP(PerceptronMulti,epochs,optimizer,criterio,input_size,data_por_lote_entrenamiento,data_por_por_lote_test)





#nuevo datos:

def escalarNuevaData(data):

    scaler=StandardScaler()

    datos_escalados=scaler.fit_transform(data)

    datos_x_tensor=torch.from_numpy(datos_escalados).float()

    return datos_x_tensor

nuevo_x=escalarNuevaData([[1,29,0,0,211,1,0,0,0,1]])

y_predict=PerceptronMulti(nuevo_x)

print(clasificador(y_predict))



Epoca[1], Perdida: 0.0630,Accuryty:0.2258
Epoca[2], Perdida: 0.0631,Accuryty:0.2204
Epoca[3], Perdida: 0.0627,Accuryty:0.2043
Epoca[4], Perdida: 0.0629,Accuryty:0.2366
Epoca[5], Perdida: 0.0630,Accuryty:0.1989
Epoca[6], Perdida: 0.0630,Accuryty:0.1828
Epoca[7], Perdida: 0.0629,Accuryty:0.1882
Epoca[8], Perdida: 0.0626,Accuryty:0.1989
Epoca[9], Perdida: 0.0631,Accuryty:0.1828
Epoca[10], Perdida: 0.0632,Accuryty:0.2258
Epoca[11], Perdida: 0.0628,Accuryty:0.1882
Epoca[12], Perdida: 0.0626,Accuryty:0.2151
Epoca[13], Perdida: 0.0628,Accuryty:0.2204
Epoca[14], Perdida: 0.0627,Accuryty:0.2151
Epoca[15], Perdida: 0.0632,Accuryty:0.2097
Epoca[16], Perdida: 0.0626,Accuryty:0.2151
Epoca[17], Perdida: 0.0626,Accuryty:0.2097
Epoca[18], Perdida: 0.0627,Accuryty:0.2258
Epoca[19], Perdida: 0.0629,Accuryty:0.1935
Epoca[20], Perdida: 0.0627,Accuryty:0.2258
Epoca[21], Perdida: 0.0626,Accuryty:0.1828
Epoca[22], Perdida: 0.0627,Accuryty:0.2043
Epoca[23], Perdida: 0.0626,Accuryty:0.2043
Epoca[24], Perdida: 

KeyboardInterrupt: ignored

In [ ]:


y_pred_multiclases = torch.tensor([[0.1, 1.8, 0.8, 0.4, 2],
                                   [0.5, 2, 3.5, 5, 2.1]])

def clasificador(y_pred_test):

   max_values_tensor = torch.empty((0,), dtype=torch.float32)

   for tensor in y_pred_test:

     max_values, max_indices  = torch.max(tensor, dim=0)

     max_values_tensor = torch.cat((max_values_tensor, max_indices.unsqueeze(0)))

   return max_values_tensor

print(clasificador(y_pred_multiclases))


tensor([4., 3.])
